Qiskit is IBM's Quantum Computing SDK, essentially granting the ability to write quantum circuits in Python rather than graphically like in the Quantum Experience.

In [2]:
# Enable IBMQ account (Feel free to make a copy and change the account key accordingly)
account_key = "" #@param {type:"string"}
!pip install qiskit
import qiskit
from qiskit import IBMQ

provider = IBMQ.enable_account(account_key)
qiskit.__version__


     |████████████████████████████████| 6.7MB 2.4MB/s 
     |████████████████████████████████| 23.3MB 1.7MB/s 
     |████████████████████████████████| 163kB 47.7MB/s 
     |████████████████████████████████| 184kB 49.0MB/s 
     |████████████████████████████████| 1.9MB 39.7MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 768kB 43.0MB/s 
     |████████████████████████████████| 5.8MB 43.2MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 296kB 28.5MB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
     |████████████████████████████████| 573kB 39.7MB/s 
     |████████████████████████████████| 26.0MB 1.3MB/s 
     |████████████████████████████████| 2.7MB 40.3MB/s 
  Created wheel for qiskit: filename=qiskit-0.19.6-cp36-none-any.whl size=3019 sha256=70defd1892c18aecb96da160ac98cf366b5894feb02d259e866d10e693a4a4a4
  Stored in directory: /root/.cache/pip/wheels/35/45/55/8b82d355edf738eb

'0.14.2'

In [3]:
import numpy as np
from qiskit import (
  QuantumCircuit,
  execute,
  Aer)
from qiskit.visualization import plot_histogram

In [4]:
# number of qubits that represent input data
NUM_DATA_QUBITS = 12

# Creating labeled data: numbers between 0 and 2^(NUM_DATA_QUBITS) - 1, checking if they're greater than 2**(NUM_DATA_QUBITS - 1)
test_training_data = []
formatStr = '{' + f'0:0>{NUM_DATA_QUBITS}b' + '}'

for i in range(1000):
  n = np.random.randint(2 ** NUM_DATA_QUBITS)
  test_training_data.append(([0 if c == "0" else 1 for c in formatStr.format(n)], 1 if n > 2 ** (NUM_DATA_QUBITS - 1) else -1))

test_training_data[:10]

[([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1], -1),
 ([1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0], 1),
 ([1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1], 1),
 ([0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0], -1),
 ([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], -1),
 ([0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1], -1),
 ([0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1], -1),
 ([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0], -1),
 ([0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1], -1),
 ([0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0], -1)]

In [6]:
from math import sqrt as msqrt

Xarray = np.asarray([
    [0, 1],
    [1, 0]
])
Zarray = np.asarray([
    [1, 0],
    [0, -1]
])

ZXarray = np.kron(Zarray, Xarray)
XXarray = np.kron(Xarray, Xarray)
ZXarray, XXarray

(array([[ 0,  1,  0,  0],
        [ 1,  0,  0,  0],
        [ 0,  0,  0, -1],
        [ 0,  0, -1,  0]]), array([[0, 0, 0, 1],
        [0, 0, 1, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0]]))

This next cell contains the class that represents the quantum neural network and its requisite operations.

In [25]:
from scipy.linalg import expm
from qiskit.providers.ibmq import least_busy
from qiskit.extensions import UnitaryGate
from qiskit.tools.monitor import job_monitor
from qiskit.quantum_info.operators import Operator


# this backend simulates a quantum computer, collapsing qubits upon measurement
backend_sim = Aer.get_backend('qasm_simulator')
# this backend gives us the statevector after computation (32 entries in this case)
statevector_backend = Aer.get_backend('statevector_simulator')
# get least busy HPC simulator
ibmq_sim_backend = least_busy(provider.backends(filters=lambda x: x.configuration().simulator))

class QNN:
  def __init__(self, num_data_qubits=12, learning_rate=0.05, params=np.random.randn(2, NUM_DATA_QUBITS)):
    self.num_data_qubits = num_data_qubits
    self.learning_rate = learning_rate
    self.params = params

  def apply_initial_state(self, circuit, sample=None):
    if sample is None:
      sample = np.random.randint(2, size=self.num_data_qubits)
    
    for i in range(self.num_data_qubits):
      if sample[i]:
        circuit.x(i)
      
    circuit.x(self.num_data_qubits)

  def generate_operators(self):
    operators = []
    for j in range(len(self.params)):
      for i in range(self.num_data_qubits):
        U_ji = (expm(1j * self.params[j, i] * ZXarray) if j == 0 else
                  expm(1j * self.params[j, i] * XXarray))

        operators.append(Operator(U_ji.tolist()))

    return operators

  def construct_circuit(self, input_data=test_training_data[0][0]):
    # this quantum circuit has 17 qubits and 1 classical register
    circ = QuantumCircuit(self.num_data_qubits + 1, 1)
    self.apply_initial_state(circ, input_data)

    for i, op_i in enumerate(self.generate_operators()):
      circ.unitary(op_i, [i % self.num_data_qubits, self.num_data_qubits], 
                   label=f"{'ZX' if i < self.num_data_qubits else 'XX'}-{self.params[i // self.num_data_qubits, i % self.num_data_qubits]}")

    # measure the results
    circ.y(self.num_data_qubits)
    circ.measure([self.num_data_qubits], list(range(circ.num_clbits)))

    return circ

  def calc_grad(self, input_index, ibm_backend):
    input_data = test_training_data[input_index][0]
    ops = self.generate_operators()
    gradient = [0.0 for i in range(len(ops))]
    circs = []
    
    for k in range(len(ops)):
      base_array = ZXarray if k < (len(ops) >> 1) else XXarray
      Sigma_k = UnitaryGate((1j * base_array).tolist()).control(1)

      grad_circ = QuantumCircuit(self.num_data_qubits + 2, 1)
      self.apply_initial_state(grad_circ, input_data)

      grad_circ.h(grad_circ.num_qubits - 1)

      # applying gates of circ but inserting Sigma_k after U_k
      for i in range(len(ops)):
        q_ind = i % self.num_data_qubits
        opGate = UnitaryGate(1j * ops[i]).control(1)
        grad_circ.append(opGate, [grad_circ.num_qubits - 1, q_ind, grad_circ.num_qubits - 2])
        if i == k:
          grad_circ.append(Sigma_k, [grad_circ.num_qubits - 1, q_ind, grad_circ.num_qubits - 2])

      grad_circ.y(grad_circ.num_qubits - 2)

      for i in range(len(ops) - 1, -1, -1):
        q_ind = i % self.num_data_qubits
        opGate = UnitaryGate(1j * ops[i].conjugate().transpose()).control(1)
        grad_circ.append(opGate, [grad_circ.num_qubits - 1, q_ind, grad_circ.num_qubits - 2])

      grad_circ.h(grad_circ.num_qubits - 1)
      grad_circ.measure([grad_circ.num_qubits - 1], list(range(grad_circ.num_clbits)))

      circs.append(grad_circ)

    # using HPC simulator
    jobs = [(i, execute(circs[i], ibm_backend, shots=1024)) for i in range(len(circs))]

    while jobs:
      for j_ind, job in jobs:
        if job.done():
          results = job.result().get_counts(circs[j_ind])
          # partial derivative is twice the imaginary component this circuit is meant to find
          smol_delta_k = 2 - 4 * results["0"] / (results["0"] + results["1"])
          gradient[j_ind] = smol_delta_k
          jobs.remove((j_ind, job))


    return np.asarray([gradient[:self.num_data_qubits], gradient[self.num_data_qubits:]])
        
  # possible labels range from 1 to -1
  @staticmethod
  def loss_function(results, data_index, debug):
    if debug:
      print("label ", test_training_data[data_index][1])
    average = (results['1'] - results['0']) / (results['0'] + results['1'])
    if debug:
      print("result ", average)
    return 1 - average * test_training_data[data_index][1]

  def update_network(self, training_index, ibm_backend):
    grad = self.calc_grad(training_index, ibm_backend)
    self.params = self.params - self.learning_rate * grad
  

What about actual measurement?

In [26]:
qnn = QNN()
circ = qnn.construct_circuit()
job_sim = execute(circ, backend_sim, shots=1024)
counts = job_sim.result().get_counts(circ)
initial_loss = QNN.loss_function(counts, 0, True)
print("initial loss ", initial_loss)
print("applying grad")
sample_indices = np.random.randint(0, len(test_training_data), len(test_training_data) // 10)
for i, train_ind in enumerate(sample_indices):
  print(f"{i}: Processing sample {train_ind}")
  qnn.update_network(train_ind, ibmq_sim_backend)
print("finished")
circ = qnn.construct_circuit()
job_sim = execute(circ, backend_sim, shots=1024)
counts = job_sim.result().get_counts(circ)
final_loss = QNN.loss_function(counts, 0, True)
print("resulting loss ", final_loss)
print("change in loss ", final_loss - initial_loss)

label  -1
result  -0.041015625
initial loss  0.958984375
applying grad
Processing sample 414
Processing sample 0
Processing sample 201
Processing sample 241
Processing sample 670
Processing sample 733
Processing sample 609
Processing sample 188
Processing sample 220
Processing sample 692
Processing sample 787
Processing sample 268
Processing sample 471
Processing sample 823
Processing sample 779
Processing sample 781
Processing sample 170
Processing sample 804
Processing sample 805
Processing sample 221
Processing sample 717
Processing sample 229
Processing sample 376
Processing sample 799
Processing sample 112
Processing sample 503
Processing sample 525
Processing sample 152
Processing sample 89
Processing sample 783
Processing sample 622
Processing sample 245
Processing sample 895
Processing sample 291
Processing sample 848
Processing sample 98
Processing sample 824
Processing sample 364
Processing sample 676
Processing sample 72
Processing sample 309
Processing sample 814
Processing

And what happens when we actually execute this circuit on one of IBM's quantum computers?

In [ ]:
# Get the least busy real quantum system
q_backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 15
                                         and not x.configuration().simulator))
print(q_backend, q_backend.status().pending_jobs)

job_q = execute(circ, q_backend, shots=1024)
job_monitor(job_q)
q_result = job_q.result()

In [ ]:
q_counts = q_result.get_counts(circ)
plot_histogram(q_counts)